In [14]:
%autoreload 2

import config
import news_handler as nh
import scrapers
import storage
import spacy

In [9]:
dummy_evt = 1
dummy_contxt = 2

articles = nh.articles_handler(dummy_evt, dummy_contxt, True)

Parsing article 0
Parsing article 1
Parsing article 2
Parsing article 3
Parsing article 4
Parsing article 5
Parsing article 6
Parsing article 7
Parsing article 8
Parsing article 9
Parsing article 10
Parsing article 11
Parsing article 12
Parsing article 13
Parsing article 14
Parsing article 15
Parsing article 16
Parsing article 17
Parsing article 18
Parsing article 19


In [15]:
# Connect to the database collection that stores the articles
store = storage.Storage(config.DB_URI)
store.connect_db(nh.DB_NAME)
store.set_collection(nh.ARTICLE_COLLECTION)

In [21]:
# Load stored articles from document store
stored_articles = store.find_all({})

# Load the english spacy model and process each article's text
nlp = spacy.load('en')
docs = [nlp(a.get('text')) for a in articles]

The document is processed through a model, which is a spaCy Pipeline consisting of a tokenizer, tagger, dependency parser and entity recognizer. Aside from the tokenizer, which receives a text string input and outputs a `Doc` object, all other steps in the pipeline apply transformations on a `Doc` object.

The `Doc` object exposes an interface of a list of Span objects, which correspond to tokens with a myriad of annotated information attributes, including pointers to other tokens in the document for dependency parses.

In [26]:
# Iterate through the document span objects and display some of the annotations as well as the subtrees
for token in docs[0][:5]:
    print(token.text, token.pos_, token.tag_, token.dep_, token.head.text, token.ent_type_)
    print("Subtree for '{}':".format(token.text))
    print(' '.join([token.text for token in token.subtree]))

# We can traverse the dep parse tree by iterating through the tokens once and checking .dep_ and .head for the 
# dependency relationship we want
from spacy.symbols import nsubj, VERB

subjects = [t for t in docs[0] if t.dep == nsubj and t.head.pos == VERB]
print("Subjects: ", subjects)

Image NOUN NN compound Image 
Subtree for 'Image':
Image
copyright NOUN NN compound Image 
Subtree for 'copyright':
copyright
Reuters PROPN NNP compound Image ORG
Subtree for 'Reuters':
Reuters
Image PROPN NNP compound caption 
Subtree for 'Image':
Image copyright Reuters Image
caption NOUN NN ROOT caption 
Subtree for 'caption':
Image copyright Reuters Image caption
Subjects:  [government, Salman, kingdom, government, Emirates, decree, citizens, state, It, personnel, Organisations, countries, %, it, Arabia, it, salaries, Arabia, this, tax]


In [27]:
# Print named entities and their categories for the first article
for ent in docs[0].ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Reuters 16 23 ORG
Saudi 42 47 NORP


State 106 113 ORG
Saudi Arabia 127 139 GPE
Salman 229 235 PERSON
monthly 248 255 DATE
more than $260 268 282 MONEY
190 285 288 MONEY
the next year 294 307 DATE
5% 382 384 PERCENT
Saudi 456 461 NORP
The United Arab Emirates 565 589 GPE
UAE 591 594 ORG
5% 618 620 PERCENT
Saudi 637 642 NORP
first 771 776 ORDINAL
Yemen 857 862 GPE
one 883 886 CARDINAL
more than $1,000 902 918 MONEY
the International Monetary Fund 943 974 ORG
Gulf 996 1000 LOC
Saudi Arabia 1076 1088 GPE
more than 90% 1089 1102 PERCENT
UAE 1162 1165 ORG
roughly 80% 1172 1183 PERCENT
2016 1189 1193 DATE
Saudi Arabia 1194 1206 GPE
last year 1278 1287 DATE
About two-thirds 1308 1324 CARDINAL
Saudis 1336 1342 NORP
45% 1422 1425 PERCENT
2015 1452 1456 DATE
Saudi Arabia 1500 1512 GPE
UAE 1521 1524 GPE
Saudi Arabia 1589 1601 GPE
UAE 1710 1713 GPE


## Matching

With `spaCy`, we can also perform rule-based matching which operates on tokens. Matchers except custom patterns described in terms of their token constituents and their attributes. A pattern is a list of dictionaries, with each dictionary being a template for one token in the pattern; a dictionary's key-value pairs act as the criteria for the token's corresponding attribute.

In [36]:
from spacy.matcher import Matcher

# Matcher instance must be initialized with a vocab
matcher = Matcher(nlp.vocab)

patterns = [{'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'}]
predicates = []
def match_callback(matcher, doc, i, matches):
    match_id, start, end = matches[i] # start and end correspond to 
    span = doc[start:end]
    span.merge()
    predicates.append(span)

matcher.add("Simple sentence", match_callback, patterns)
matches = matcher(docs[1])

print(predicates)

[]


In [38]:
a = articles[0]
a.get('url')

'http://www.bbc.co.uk/news/world-middle-east-42587037'

In [39]:
from newspaper import Article

article = Article(a.get('url'))
article.build()
article.summary

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/roland/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************